# Congressional Speech Analysis

### 1. Data import

In [1]:
import pandas as pd
# its necessary to run --> pip3 install xlrd==1.2.0               
df_charisma_analysis=pd.read_excel("charisma_analisis_complete_df.xlsx", header=0)
df_congress_114_bills=pd.read_excel("congress_114_bills_df_new.xlsx",header=0)


---
### 2. Cleaning data

##### 2.1 Indices 

In [2]:
print(f'''For df_congress_114_bills the field sentence_id has {sum(df_congress_114_bills.sentence_id.value_counts()>1)} repeted values\n''')
print(f'''While for df_charisma_analysis the field paragraph_id has {sum(df_charisma_analysis.paragraph_id.value_counts()==2)} repeted index''')

For df_congress_114_bills the field sentence_id has 0 repeted values

While for df_charisma_analysis the field paragraph_id has 24 repeted index


---
### 3. Following sugestions

##### 1 - Merging the bills dataframe and the complete dataframe of scored paragraphs by the unique_speech_id column

In [3]:
print(f'''The unique_speech_id column in df_charisma_analysis has 
     {sum(df_charisma_analysis.unique_speech_id.value_counts()[df_charisma_analysis.unique_speech_id.value_counts()>1])} rows repeated''')
print(f'''     That is {sum(df_charisma_analysis.unique_speech_id.value_counts()[df_charisma_analysis.unique_speech_id.value_counts()>1])/df_charisma_analysis.shape[0]:.0%} of the rows repeated\n''')

print(f'''The unique_speech_id column in df_congress_114_bills has 
     {sum(df_congress_114_bills.unique_speech_id.value_counts()[df_congress_114_bills.unique_speech_id.value_counts()>1])} rows repeated''')

print(f'''     That is {sum(df_congress_114_bills.unique_speech_id.value_counts()[df_congress_114_bills.unique_speech_id.value_counts()>1])/df_congress_114_bills.shape[0]:.0%} of the rows repeated\n''')


The unique_speech_id column in df_charisma_analysis has 
     8353 rows repeated
     That is 76% of the rows repeated

The unique_speech_id column in df_congress_114_bills has 
     2278 rows repeated
     That is 59% of the rows repeated



If we merge the data using **unique_spech_id** the result will be:

In [4]:
df_merge_1=pd.merge(df_congress_114_bills, df_charisma_analysis,on='unique_speech_id')
print(f'''A data base with {df_merge_1.shape[0]} rows, where the unique_speech_id has {sum(df_merge_1.unique_speech_id.value_counts()[df_merge_1.unique_speech_id.value_counts()>1])} rows repeted''')

A data base with 1005 rows, where the unique_speech_id has 972 rows repeted


##### 2 - Merging the bills dataframe and the complete dataframe of scored paragraphs by the document_id and speech_id columns simultaneously 

In the case of **df_congress_114_bills** the index created by *document_id* and *speech_id*:

In [5]:
ind_1= df_congress_114_bills.document_id.astype(str)+'-'+df_congress_114_bills.speech_id.astype(str)
print(f'''produce {sum(ind_1.value_counts()[ind_1.value_counts()>1])} rows repeated of {df_congress_114_bills.shape[0]} ''')

produce 2278 rows repeated of 3886 


In the case of **df_charisma_analysis** the index created by *document_id* and *speech_id*:

In [6]:
ind_2= df_charisma_analysis.document_id.astype(str)+'-'+df_charisma_analysis.speech_id.astype(str)
print(f'''produce {sum(ind_2.value_counts()[ind_2.value_counts()>1])} rows repeated of {df_charisma_analysis.shape[0]} ''')

produce 8353 rows repeated of 10938 


When the two indexes cross:

In [7]:
ind_l1=set(ind_1.values)
ind_l2=set(ind_2.values)
print(f'''There are {sum([ a in ind_l2 for a in ind_l1])} elements of df_congres_114_bills index_adhoc in df_charisma_analysis''')

There are 123 elements of df_congres_114_bills index_adhoc in df_charisma_analysis


In [12]:
# generating the merging database:
df_congress_114_bills['index_adhoc']= df_congress_114_bills.document_id.astype(str)+'-'+df_congress_114_bills.speech_id.astype(str)
df_charisma_analysis['index_adhoc']= df_charisma_analysis.document_id.astype(str)+'-'+df_charisma_analysis.speech_id.astype(str)
df_merge_2=pd.merge(df_congress_114_bills,df_charisma_analysis, on='index_adhoc')
print(f'''A data base with {df_merge_2.shape[0]} rows, where the index_adhoc has {sum(df_merge_2.index_adhoc.value_counts()[df_merge_2.index_adhoc.value_counts()>1])} rows repeted''')

A data base with 1005 rows, where the index_adhoc has 972 rows repeted


##### 3 - Try looking only at the document_id and check that the same document_id exists in both the paragraphs and the sentences dataframes. 

In [49]:
doc_id_congress=[doc for doc in df_congress_114_bills.document_id.values] # creating list of doc_id from db_congress
doc_id_db_congress_in_db_charisma=[df_charisma_analysis[df_charisma_analysis.document_id==ele].shape[0] for ele in doc_id_congress]
doc_id_db_congress_in_db_charisma=pd.Series(data=doc_id_db_congress_in_db_charisma)
doc_id_db_congress_in_db_charisma.value_counts().sum

print(f'''document_id from db_congress_114 in db_charisma 
      {len(doc_id_db_congress_in_db_charisma)-doc_id_db_congress_in_db_charisma.value_counts()[0]} 
      of which {doc_id_db_congress_in_db_charisma.value_counts()[1]} are unique''')

document_id from db_congress_114 in db_charisma 
      1970 
      of which 273 are unique


In [53]:
doc_id_charisma=[doc for doc in df_charisma_analysis.document_id.values] # creating list of doc_id from db_congress
doc_id_db_charisma_in_db_congress=[df_congress_114_bills[df_congress_114_bills.document_id==ele].shape[0] for ele in doc_id_charisma]
doc_id_db_charisma_in_db_congress=pd.Series(data=doc_id_db_charisma_in_db_congress)
doc_id_db_charisma_in_db_congress.value_counts().sum

print(f'''document_id from db_charisma in db_congress_114 
      {len(doc_id_db_charisma_in_db_congress)-doc_id_db_charisma_in_db_congress.value_counts()[0]} 
      of which {doc_id_db_charisma_in_db_congress.value_counts()[1]} are unique''')

document_id from db_charisma in db_congress_114 
      2976 
      of which 956 are unique


In [56]:
df_merge_3=pd.merge(df_charisma_analysis,df_congress_114_bills,on='document_id')

In [58]:
df_merge_3.shape

(23053, 201)